# LWE Grundlagen

Die Folgenden abschnitte sind zur Beschreibung und einführung in das LWE Problem. Dies ist die Grundlage für verschiedene moderne Verschlüsselungsalgorithmen wie Kyber, aber auch die basis für diverse Homomorphe Algorithmen.

Im folgenden werden 3 Arten von LWE beschrieben und implementiert
- Learning with Errors (LWE)
- Ring-LWE (RLWE)
- Module-LWE (MLWE)

Das LWE verfahren liefert dabei die Grundlagen auf denen die anderen beiden Aufbauen. Bei RLWE wird das LWE verfahren auf einen Polynom Ring übertragen und bei MLWE wird eine Matrix aus Polynom Ringen benutzt. RLWE ist somit ein Spezialfall von MLWE, bei dem eine 1x1 Matrize verwendet wird.

Zur Umsetzung von LWE muss zuerst ein Ring der ganzen zahlen modulo eines Faktors $q$ definiert werden, welcher die addition (+) und multiplikation (*) unterstützt Dies wird definiert als 
$$\mathbb{Z}_q= \mathbb{Z}/q\mathbb{Z}$$
Dies ist somit eigentlich nur die Gruppe aller ganzen Zahlen, welche kleiner als $q$ und größer gleich 0 sind. Beispielsweise $\mathbb{Z}_3 = \{0,1,2\}$

## LWE

Das LWE Problem lässt sich auf folgende Gleichung herunterbrechen: 
$$A⋅s+e=b$$
mit $A \in \mathbb{Z}_q^{n\times m}$, $s \in \mathbb{Z}_q^{m}$ welche gleichmäßig aus $\mathbb{Z}_q$ gezogen werden und $e \in \mathbb{Z}_q^n$ entnommen aus der diskreten Gaußverteilung sodass die wahrscheinlichkeit das die Werte kleiner sind als $q/4$ sehr hoch ist (im folgenden $\chi$). Daraus folgt das $b \in \mathbb{Z}_q^{n}$

Private key: $s\leftarrow \mathbb{Z}_q^{m}$. Dabei entsteht ein vektor mit länge $m$ bei dem alle werte gleichmäßig zufällig aus $\mathbb{Z}_q$ stammen.

Public key:
1. erstellen einer Matrix mit gleichmäßig zufälligen werten: 
$A = \begin{bmatrix}
a_11 & \cdots  & a_1n\\
\vdots  & \ddots  & \vdots \\
a_m1 & \cdots & a_mn
\end{bmatrix} \leftarrow \mathbb{Z}_q^{n \times m} $
2. berechnen des vektors $b$ durch: $b = As+e$ mit $e \leftarrow \chi^n $
3. der private schlüsse ergibt sich aus den zwei Werten: $P = (A, b)$

Zum Verschlüsseln eines binären Wertes $m \in \mathbb{Z}_2$ (entspricht den Werten $\{0, 1\}$) werden zwei Werte Berechnet:
1. $u = A^T \cdot r + e_1 \in \mathbb{Z}_q^m$
2. $v = b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor) \in \mathbb{Z}_q$

Wobei $r \in \mathbb{Z}_q^n$ zusätzliche Werte sind um ein zusätzliche Verschleierung zu erzeugen und $e_1 \in \chi^m$ und $e_2 \in \chi$ sind zusätzliche Fehlerwerte. Durch diese drei werte wird jede Verschlüsselung einzigartig und somit schwerer zu entschlüsseln durch das sammeln vieler verschlüsselter Texte. 

Somit ergeben sich zwei werte, der Vektor $u$ und der skalar $v$. Die Nachricht wurde dabei auf skaliert sodas $0 \rightarrow 0$ und $1 \rightarrow \left\lfloor q/2\right\rfloor$. Wenn man sich den Ring $\mathbb{Z}_q$ als Uhr dabei vorstellt (wie es oft für modulo Rechnungen visualisiert wird), dann ist der Nachrichten Wert $0$ nun an der 12Uhr position in der Uhr und der Nachrichten Wert $1$ (ungefähr) an der 6 Uhr Position. Durch die einberechneten Fehler verändert sich der eigentliche Nachrichten Wert um ein kleines bisschen, sodass er quasi mehr in die viertel oder dreiviertel Stellung auf der Uhr zeigt.

Das entschlüsseln der Nachricht erfolgt mithilfe folgender Gleichung:
$$m = \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(v-s^T \cdot u)\right\rceil _2 \in \{0,1\}$$
$$
\begin{align*}
m &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(v-s^T \cdot u)\right\rceil _2 \\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor)-s^T \cdot (A^T \cdot r + e_1))\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*((As+e)^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor)-s^T A^T \cdot r - s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*((As)^T \cdot r + e^Tr+ e_2 + (m*\left\lfloor q/2\right\rfloor)-(As)^T \cdot r - s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(e^Tr+ e_2 + (m*\left\lfloor q/2\right\rfloor)- s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{e^Tr}{\left\lfloor q/2\right\rfloor}+ \frac{e_2 }{\left\lfloor q/2\right\rfloor}+ m - \frac{s^T e_1}{\left\lfloor q/2\right\rfloor}\right\rceil _2\\
  &= \left\lfloor m' \right\rceil _2\\
  &= m
\end{align*}
$$

Bei den letzten zwei schritten wird davon ausgegangen das die Fehlerwerte nahe null sind und somit nur einen geringen einfluss auf $m$ haben. Dadurch werden diese Werte durch das runden wieder ausgeglichen und die Nachricht kommt am ende zum vorschein, welche noch in ihre ring (modulo 2) angepasst werden muss.

In [19]:
from scipy.stats import norm
import numpy as np

def sample_lwe_error(n, q, sigma=1.0):
  """Samples error vector from an approximated discrete Gaussian distribution.

  Args:
      n: Dimension of the error vector.
      q: Modulus for the ring (positive integer).
      sigma: Standard deviation of the Gaussian distribution (default 1.0).

  Returns:
      A numpy array representing the error vector modulo q.
  """
  # Sample from continuous Gaussian with mean 0 and std dev sigma
  return np.round(norm.rvs(loc=0, scale=sigma, size=n)).astype(int) % q 

# Example usage
n = 10
q = 50
sigma = 1
error = sample_lwe_error(n, q, sigma)
print(error)

[ 0 49  0  1 49 49  1  1 49 49]
